# Abstract
In this report we analyze summary statistic for a number of US equitites and make recommendations based on three different sets of portfolio constraints

# Portfolio A: Long 1 / Short 1

# Portfolio B: Long Only Portfolio

# Portfolio C: Long/Short Three or More Total Positions

# Returns Statistics for Tradable Universe

In [17]:
import pandas as pd
stats_daily = pd.read_csv("./MScFE560_FM_GWP1_Data.csv",index_col=0).iloc[:4,:4].astype(float)
corr_daily = pd.read_csv("./MScFE560_FM_GWP1_Data.csv",index_col=0).iloc[6:,:5].astype(float)